In [461]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from ta.trend import SMAIndicator
import yfinance as yf
import pandas as pd
import json

import numpy as np

class finance_manager:
    def __init__(self) -> None:
        self.data = {}

    def get_data(self,company_id,meashure,_start_period,_end_period):
        self.data[company_id]=yf.download(company_id,interval=meashure,start=_start_period,end=_end_period)
        self.data[company_id] = self.data[company_id].fillna(method='bfill')
        self.data[company_id] = self.data[company_id].fillna(method='ffill')

    def print_data(self,company_id,*param):
        temp_df = self.data[company_id].copy()
        fig = go.Figure()
        for item in param:
            if("position" in item):
                fig.add_trace(go.Scatter(x=temp_df.index,y=temp_df[item]*60,name=item))
            elif("ret_acum" in item):
                fig.add_trace(go.Scatter(x=temp_df.index,y=temp_df[item]*100,name=item))
            else:
                fig.add_trace(go.Scatter(x=temp_df.index,y=temp_df[item],name=item))
                
        fig.show()

class khalman_signal_filter:
    def __init__(self,new_info=0.5,old_info=0.5):
        self.y_n=1
        self.y_n1=1
        self.new_info = new_info
        self.old_info = old_info

    def filter(self,Data:pd.DataFrame):
        temp_vector = []
        for x_0 in Data:
            temp_val = self.y_n
            self.y_n = self.new_info*x_0 + self.old_info*self.y_n1
            self.y_n1 = temp_val
            temp_vector.append(self.y_n)
        return temp_vector
        
    def reset(self):
        self.y_n=1
        self.y_n1=1

class exponential_signal_filter:
    def __init__(self,_a=0.5):
        self.y_n=1
        self.y_n1=1
        self.a = _a

    def filter(self,Data:pd.DataFrame):
        temp_vector = []
        for x_0 in Data:
            temp_val = self.y_n
            self.y_n = self.a*self.y_n1+(1-self.a)*x_0
            self.y_n1 = temp_val
            temp_vector.append(self.y_n)
        return temp_vector
    def reset(self):
        self.y_n=1
        self.y_n1=1

def function_retard(data,index):
    return data.shift(index)

def get_position(x_low,x_high):
    temp_vector=[]
    for index in range(len(x_low)):
        if x_low[index]*0.99>x_high[index]:
            temp_vector.append(1)
        else:
            temp_vector.append(0)
    return temp_vector

In [462]:
company_id = "ETC-USD"
fm = finance_manager()
KF = khalman_signal_filter(0.25,0.75)
EF = exponential_signal_filter(0.8)
fm.get_data(company_id,"1d",'2020-01-01','2022-01-01')
fm.data[company_id]["Close Khalman"]= KF.filter(fm.data[company_id]["Close"])
fm.data[company_id]["Close Khalman Exponential"]= EF.filter(fm.data[company_id]["Close Khalman"])
fm.data[company_id]["Close Khalman Retard"]= function_retard(fm.data[company_id]["Close Khalman"],5)
fm.data[company_id]["Close KE position"]=get_position(fm.data[company_id]["Close Khalman"],fm.data[company_id]["Close Khalman Exponential"])
fm.data[company_id]["Close KR position"]=get_position(fm.data[company_id]["Close Khalman"],fm.data[company_id]["Close Khalman Retard"])
# fm.print_data(company_id,"Close","Close Khalman","Close Khalman Exponential")

fm.data[company_id]['retornos_del_instrumento'] = fm.data[company_id]['Close'].pct_change()
fm.data[company_id]['estrategia1'] = fm.data[company_id]['Close KE position'].shift(1)*fm.data[company_id]['retornos_del_instrumento']
fm.data[company_id]['estrategia2'] = fm.data[company_id]['Close KR position'].shift(1)*fm.data[company_id]['retornos_del_instrumento']
fm.data[company_id]['ret_acum'] = fm.data[company_id]['retornos_del_instrumento'].cumsum().apply(np.exp)
fm.data[company_id]['ret_acum_est2'] = fm.data[company_id]['estrategia2'].cumsum().apply(np.exp)
fm.data[company_id]['ret_acum_est1'] = fm.data[company_id]['estrategia1'].cumsum().apply(np.exp)

# fm.print_data(company_id,"Close","Close Khalman","Close Khalman Exponential","Close Khalman Retard")
fm.print_data(company_id,"Close","Close Khalman","Close KE position","ret_acum","ret_acum_est2")
# fm.print_data(company_id,"ret_acum","ret_acum_est")


[*********************100%***********************]  1 of 1 completed
